In [1]:
from rdkit import Chem
from rdkit.Chem import Descriptors, Crippen, rdMolDescriptors, QED

In [2]:
def compute_properties(smiles):
    if isinstance(smiles, list):
        return [compute_properties(s) for s in smiles]

    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None

    return {
        "AromaticRings": rdMolDescriptors.CalcNumAromaticRings(mol),
        "MW": Descriptors.MolWt(mol),
        "LogP": Crippen.MolLogP(mol),
        "Fsp3": rdMolDescriptors.CalcFractionCSP3(mol),
        "QED": QED.qed(mol),
    }

def print_properties(smiles_list):
    for smi in smiles_list:
        props = compute_properties(smi)

        if props is None:
            print(f"{smi} | INVALID")
            continue

        print(
            f"{smi} | "
            f"AromaticRings={props['AromaticRings']} | "
            f"MW={props['MW']:.2f} | "
            f"LogP={props['LogP']:.2f} | "
            f"Fsp3={props['Fsp3']:.2f} | "
            f"QED={props['QED']:.3f}"
        )

In [3]:
smiles_list = [
    "CN(C)c1ccc2c(c1)CC1=CC(=[N+](C)C)C(=N2)C=C1", # input
    "CN(C)c1ccc2c(c1)CC1=CC(=C(C)C)[C@H](C2=O)C=N1", # output
    "COc1cc(N)c2c(c1N)C(=O)c1ccccc1C2=O", # input
    "COc1cc(N)c2c(c1)NC(=O)c1ccccc1C2=O", # output
    "Cc1cc(C=Cc2ccc(N(C)C)cc2)cc(C)[o+]1", # input
    "Cc1cc(C=Cc2ccc(N(C)C)cc2)cc(C)n1", # output
    "O=[N+]([O-])c1ccc(Cc2ccncc2)cc1", # input
    "O=S([O-])c1ccc(Cc2ccncc2)cc1"] # output

In [4]:
print_properties(smiles_list)

CN(C)c1ccc2c(c1)CC1=CC(=[N+](C)C)C(=N2)C=C1 | AromaticRings=1 | MW=266.37 | LogP=2.59 | Fsp3=0.29 | QED=0.564
CN(C)c1ccc2c(c1)CC1=CC(=C(C)C)[C@H](C2=O)C=N1 | AromaticRings=1 | MW=280.37 | LogP=3.41 | Fsp3=0.33 | QED=0.789
COc1cc(N)c2c(c1N)C(=O)c1ccccc1C2=O | AromaticRings=2 | MW=268.27 | LogP=1.64 | Fsp3=0.07 | QED=0.654
COc1cc(N)c2c(c1)NC(=O)c1ccccc1C2=O | AromaticRings=3 | MW=268.27 | LogP=1.63 | Fsp3=0.07 | QED=0.657
Cc1cc(C=Cc2ccc(N(C)C)cc2)cc(C)[o+]1 | AromaticRings=2 | MW=254.35 | LogP=4.41 | Fsp3=0.24 | QED=0.753
Cc1cc(C=Cc2ccc(N(C)C)cc2)cc(C)n1 | AromaticRings=2 | MW=252.36 | LogP=3.93 | Fsp3=0.24 | QED=0.822
O=[N+]([O-])c1ccc(Cc2ccncc2)cc1 | AromaticRings=2 | MW=214.22 | LogP=2.58 | Fsp3=0.08 | QED=0.582
O=S([O-])c1ccc(Cc2ccncc2)cc1 | AromaticRings=2 | MW=232.28 | LogP=1.91 | Fsp3=0.08 | QED=0.759


In [5]:
import sys
from rdkit import Chem

# Adjust path to where you saved sascorer.py
sys.path.append("/Users/md_halim_mondol/Desktop/RDkit_practice")

import sascorer  # from Contrib/SA_Score

def sa_score_from_smiles(smiles: str):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    return float(sascorer.calculateScore(mol))

# Input SMILES
print("SA Scores of Input SMILES:")
print(sa_score_from_smiles("CN(C)c1ccc2c(c1)CC1=CC(=[N+](C)C)C(=N2)C=C1"))
print(sa_score_from_smiles("COc1cc(N)c2c(c1N)C(=O)c1ccccc1C2=O"))
print(sa_score_from_smiles("Cc1cc(C=Cc2ccc(N(C)C)cc2)cc(C)[o+]1"))
print(sa_score_from_smiles("O=[N+]([O-])c1ccc(Cc2ccncc2)cc1"))

print(50*'-')

# Reconstructed SMILES
print("SA Scores of Reconstructed SMILES:")
print(sa_score_from_smiles("CN(C)c1ccc2c(c1)CC1=CC(=C(C)C)[C@H](C2=O)C=N1"))
print(sa_score_from_smiles("COc1cc(N)c2c(c1)NC(=O)c1ccccc1C2=O"))
print(sa_score_from_smiles("Cc1cc(C=Cc2ccc(N(C)C)cc2)cc(C)n1"))
print(sa_score_from_smiles("O=S([O-])c1ccc(Cc2ccncc2)cc1"))

SA Scores of Input SMILES:
4.327671049179102
2.2174763548319962
2.731735735977347
1.7501454653148887
--------------------------------------------------
SA Scores of Reconstructed SMILES:
4.626392301334721
2.217146564622208
2.246234720012181
2.9924217803814024


In [6]:
from rdkit import Chem
from rdkit.Chem import Draw
import os

def draw_smiles_list_2d(
    smiles_list,
    out_dir="molecules_2d",
    img_size=(300, 300),
    bond_width=2,
    font_size=0.8,
    prefix="mol"
):
    os.makedirs(out_dir, exist_ok=True)

    for i, smi in enumerate(smiles_list):
        mol = Chem.MolFromSmiles(smi)
        if mol is None:
            print(f"[WARN] Invalid SMILES at index {i}: {smi}")
            continue

        # Compute 2D coordinates
        Chem.rdDepictor.Compute2DCoords(mol)

        # Drawer
        drawer = Draw.MolDraw2DCairo(img_size[0], img_size[1])
        opts = drawer.drawOptions()
        opts.bondLineWidth = bond_width
        opts.fontSize = font_size

        drawer.DrawMolecule(mol)
        drawer.FinishDrawing()

        png = drawer.GetDrawingText()
        fname = os.path.join(out_dir, f"{prefix}_{i:03d}.png")

        with open(fname, "wb") as f:
            f.write(png)

        print(f"[OK] Saved: {fname}")

In [7]:
draw_smiles_list_2d( smiles_list, out_dir="dye", img_size=(320, 320), bond_width=2, font_size=0.85, prefix="latent" )

[OK] Saved: dye/latent_000.png
[OK] Saved: dye/latent_001.png
[OK] Saved: dye/latent_002.png
[OK] Saved: dye/latent_003.png
[OK] Saved: dye/latent_004.png
[OK] Saved: dye/latent_005.png
[OK] Saved: dye/latent_006.png
[OK] Saved: dye/latent_007.png
